In [2]:
import pandas as pd
import os
from src.utils.results_utils import mapper
from fuzzywuzzy import fuzz, process

In [3]:
folder = "static"
df = pd.DataFrame()
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        try:
            temp = pd.read_excel(os.path.join(folder, file), sheet_name="Data")
        except:
            temp = pd.read_excel(os.path.join(folder, file), sheet_name="Sheet 1")
        df = pd.concat([df, temp])


In [4]:
df.ffill(inplace=True)

In [25]:
target = df[["Maid’s ID", "Modified Field", "Maid’s Nationality", "Agent Value"]]

In [26]:
places = target[target["Modified Field"] == "Birth Place"]

In [34]:
# Get unique places and nationalities
places_unique = places[["Maid’s ID", "Maid’s Nationality", "Agent Value"]].drop_duplicates(subset=["Maid’s Nationality", "Agent Value"])

# Normalize nationality to country code using mapper
def get_country_code(nationality):
    # Use fuzzy matching to find the closest key in mapper
    matched_country, score = process.extractOne(nationality, mapper.keys())
    return mapper.get(matched_country, "XXX")

places_unique["country_code"] = places_unique["Maid’s Nationality"].apply(get_country_code)

# Prepare DataFrame for custom_cities.csv format
places_to_add = places_unique[["Agent Value", "country_code", "Maid’s ID"]].rename(columns={"Agent Value": "name"})
place_ids = places_to_add[['name', 'Maid’s ID']]
place_ids = place_ids.groupby('name').agg(lambda x: list(x)).reset_index()

# Load existing custom_cities.csv
custom_cities_path = os.path.join(folder, "custom_cities.csv")
if os.path.exists(custom_cities_path):
    existing_cities = pd.read_csv(custom_cities_path)
else:
    existing_cities = pd.DataFrame(columns=["name", "country_code"])

# Find new places that are not in the existing custom_cities.csv
existing_places = set(existing_cities["name"].str.upper())
places_to_add["name"] = places_to_add["name"].str.upper()
new_places = places_to_add[~places_to_add["name"].isin(existing_places)]

# Combine existing and new places
combined_cities = pd.concat([existing_cities, new_places]).drop_duplicates(subset=["name"])

# Save updated custom_cities.csv
combined_cities.to_csv(custom_cities_path, index=False)

# Display how many new places were added
len(new_places)

                 name  Maid’s ID
0      A CASTANEDA NV  [87863.0]
1      A LISTA IFUGAO  [81935.0]
2             ABAREBO  [82048.0]
3             ABEGORA  [80776.0]
4               ABELA  [84834.0]
...               ...        ...
4134            ZIWAY  [82072.0]
4135  ZUMARRAGA SAMAR  [87356.0]
4136       ZVISHAVANE  [86988.0]
4137   Zamboanga City  [75298.0]
4138            Ziway  [76259.0]

[4139 rows x 2 columns]


0

In [42]:
# Identify similar city names within each country code
from collections import defaultdict

# Set similarity threshold
similarity_threshold = 95  # Adjust as needed (higher = stricter matching)

# Group places by country code
country_groups = places_to_add.groupby('country_code')

# Dictionary to store similar place clusters for each country
similar_places = defaultdict(list)

# Process each country's places
for country_code, group in country_groups:
    places_list = group['name'].tolist()
    
    # Skip processing if too few places
    if len(places_list) < 2:
        continue
    
    # Find similar places within this country
    processed = set()
    
    for i, place1 in enumerate(places_list):
        if place1 in processed:
            continue
            
        cluster = [place1]
        processed.add(place1)
        
        for place2 in places_list[i+1:]:
            if place2 in processed:
                continue
                
            # Calculate similarity ratio
            similarity = fuzz.ratio(place1, place2)
            
            # If similar enough, add to cluster
            if similarity >= similarity_threshold:
                cluster.append(place2)
                processed.add(place2)
        
        # Store cluster if it contains multiple places
        if len(cluster) > 1:
            similar_places[country_code].append(cluster)

place_counts = places_to_add['name'].value_counts().to_dict()

# Create a DataFrame with all similar place clusters for easier inspection
similar_df_rows = []
country = "PHL"
clusters = similar_places[country]

for cluster in clusters:
    for place in cluster:
        print(place, place_counts.get(place, 0), place_ids[place_ids['name'] == place]['Maid’s ID'].tolist(), end=" ")
    print()

G GENEROSO DV OR 1 [[85982.0]] G GENEROSO DVO OR 1 [[75564.0]] 
SN FERNANDO CEBU 1 [[77522.0]] SAN FERNANDO CEBU 1 [[89837.0]] 
MATALAM NO COT 1 [[86862.0]] MATALAM N COT 1 [[88297.0]] 
SANCHEZ MIRA CAG 1 [[86967.0]] SANCHEZ MIRA CAGE 1 [[92705.0]] 
ALLANERA N ECIJA 1 [[86928.0]] LLANERA N ECIJA 1 [[84366.0]] 
KIDAPAWAN NO COT 1 [[84771.0]] KIDAPAWAN NO CO 1 [[91712.0]] 
SAN FERNANDO PMP 1 [[85775.0]] SAN FERNANDO PAMP 1 [[87102.0]] 
SAN LUIS PAMPANGA 1 [[87416.0]] SN LUIS PAMPANGA 1 [[85282.0]] 
LAGUINDINGAN MO 1 [[85280.0]] LAGUINDINGAN M O 1 [[86057.0]] 
BANGA S COTABATO 1 [[84942.0]] BANGA SO COTABATO 1 [[91688.0]] 
GAPAN N ECIJA 1 [[80319.0]] GAPAN NECIJA 1 [[86087.0]] 
KALIBO AKLAN 1 [[80321.0]] KALIBO KLAN 1 [[80321.0]] 
VALENCIA BKN 1 [[82897.0]] VALECIA BKN 1 [[93211.0]] 
MINGLANILA CEBU 1 [[80334.0]] MINGLANILLA CEBU 1 [[84856.0]] 
LAPU LAPU CEBU 2 [[84450.0]] LAPULAPU CEBU 1 [[81000.0]] 
APARRI CAGAYAN 1 [[87029.0]] APARRI AGAYAN 1 [[82652.0]] 
DUPAX DN N VIZ 2 [[83243.0]] D

In [ ]:
# After reviewing similar_places.csv, create a mapping of misspelled names to correct names
# For example:
corrections = {
    # 'MISSPELLED': 'CORRECT',
    # 'NAIROBE': 'NAIROBI',
    # Add your corrections here
    "ADDIS ABEBA": "ADDIS ABABA",
    "NABUNTURAN DDN": "NABUNTURAN DON",
    "DAGUPAN CI": "DAGUPAN CITY",
    "STO TOMAS DVO": "STO TOMAS DAVAO",
    "STO TOMAS DVO DN": "STO TOMAS DAVAO",
    "STO TOMAS DAVAD": "STO TOMAS DAVAO",
    "STO TOMAS DVD": "STO TOMAS DAVAO",
    "S-BULLONES BOHO": "S BULLONES BOHO",
}

# Apply corrections to the places_to_add DataFrame
places_to_add['name'] = places_to_add['name'].apply(lambda x: corrections.get(x, x))

# Remove duplicates after corrections
places_to_add = places_to_add.drop_duplicates(subset=['name', 'country_code'])

# Update combined_cities with corrected names
combined_cities = pd.concat([existing_cities, places_to_add]).drop_duplicates(subset=['name'])

# Save updated custom_cities.csv
combined_cities.to_csv(custom_cities_path, index=False)

print(f"Updated custom_cities.csv with corrections")